pip install nest_asyncio

git clone --depth=1 https://github.com/twintproject/twint.git<br>
cd twint<br>
pip3 install . -r requirements.txt<br>

+ aiohttp==3.7.0 setzen in requirements.txt

In [1]:
import twint
import pandas as pd
import datetime
import schedule
import time
import os
import nest_asyncio
nest_asyncio.apply()
pd.set_option('display.max_columns', None)

c = twint.Config()

In [2]:
# Erstellt aus String eine Liste
from ast import literal_eval
import re
def extract_reply_to(stringliste):
    if len(stringliste) > 10:
        mentionsliste = []
        liste = literal_eval(stringliste)
        for mention in liste:
            mentionsliste.append(mention["screen_name"])
        return mentionsliste
    else:
        return stringliste

In [3]:
def searchTwint(keyword, since=None, until=None, language=None, limit=None, createDataFrame=True, hide_out=True):
    c = twint.Config()
    c.Search = "{} lang:{}".format(keyword, language)
    c.Filter_retweets = False
    if language is not None:
        c.Lang = language
    if since is not None:
        c.Since = since
    if until is not None:
        c.Until = until
    if limit is not None:
        c.Limit = limit
    c.Hide_output = hide_out
    if createDataFrame:
        c.Pandas = True
    twint.run.Search(c)
    if createDataFrame:
        Tweets_df = twint.storage.panda.Tweets_df
        return Tweets_df

In [16]:
df = searchTwint("corona", since="2022-05-18", until="2022-05-19", language="de", limit=100, createDataFrame=True)

In [17]:
# Duplikate werden entfernt
df = df.drop_duplicates(subset=["id"])
df.drop(columns=["created_at", "place", "cashtags", "user_id_str", "day", "hour", "urls", "thumbnail", "quote_url", "near", "geo", "source", "user_rt_id", "user_rt", "retweet_id", "retweet", "retweet_date", "translate", "trans_src", "trans_dest"], inplace=True)

In [18]:
df = df[df["language"] == "de"]
df["date"] = df["date"].astype("datetime64")
#df["created_at"] = df["created_at"].astype("datetime64")
df["Wochentag"] = df["date"].apply(lambda x: x.weekday())
df["Stunde"] = df["date"].apply(lambda x: x.hour)
df["Originale_Tweetlaenge"] = df["tweet"].apply(lambda x: len(x))
df["reply_to"] = df["reply_to"].apply(lambda x: str(extract_reply_to(x)))

In [19]:
df

,id,conversation_id,date,timezone,tweet,language,hashtags,user_id,username,name,link,photos,video,nlikes,nreplies,nretweets,search,reply_to,Wochentag,Stunde,Originale_Tweetlaenge
0,1527075828277952515,1526948275726729217,2022-05-19 01:57:03,+0200,@WuChenFu3 @DrNathanCole Manchmal ist er ganz ...,de,[],1238892997535825920,HN_112_,🇺🇦 Polonaise mit Abstand & mit enthemmten Mob 🧹,https://twitter.com/HN_112_/status/15270758282...,[https://pbs.twimg.com/media/FTFDX1mWIAUh1Ro.jpg],1,1,0,0,corona lang:de,"[{'screen_name': 'WuChenFu3', 'name': 'WuChenF...",3,1,258
1,1527075720224284672,1527075720224284672,2022-05-19 01:56:38,+0200,New post (Corona: Warum sich manche noch nicht...,de,[],43948964,ATGBrokers,Alexander,https://twitter.com/ATGBrokers/status/15270757...,[],0,0,0,0,corona lang:de,[],3,1,138
2,1527075580121862145,1527075580121862145,2022-05-19 01:56:04,+0200,Corona Pandemiestufe 3 - wie viele Haushalte d...,de,[],705825629120405509,malnefrage,malnefrage.de,https://twitter.com/malnefrage/status/15270755...,[],0,0,0,0,corona lang:de,[],3,1,204
3,1527075577852747778,1527075577852747778,2022-05-19 01:56:04,+0200,Sinnlose PCR-Tests: ➡️ Australische Behörde g...,de,"[covid19, sarscov2, corona, pcrtests]",284182005,Roellchen2011,Susanne*Roellchen,https://twitter.com/Roellchen2011/status/15270...,[],0,2,0,2,corona lang:de,[],3,1,143
4,1527075324516831232,1526868523825475584,2022-05-19 01:55:03,+0200,@womawo @ABaerbock @Minister1053 Hören Sie auf...,de,[],1451534855242719232,Palindromaviva,Palindrom,https://twitter.com/Palindromaviva/status/1527...,[],0,2,0,0,corona lang:de,"[{'screen_name': 'womawo', 'name': '🇺🇦 Marita'...",3,1,224
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,1527052854669746176,1526922017437634563,2022-05-19 00:25:46,+0200,@Kraeuselhirn Hatte das schon lange nicht mehr...,de,[],3253229458,achtuhrkatze,krauchet hatz,https://twitter.com/achtuhrkatze/status/152705...,[],0,1,0,0,corona lang:de,"[{'screen_name': 'Kraeuselhirn', 'name': 'DerM...",3,0,243
96,1527052782146052097,1527050674055401472,2022-05-19 00:25:29,+0200,@Cyhra1 Der ist richtig hartleibig. Wenns mit ...,de,[],1038186234437398528,schonalthalt,schonalthalt,https://twitter.com/schonalthalt/status/152705...,[],0,2,0,0,corona lang:de,"[{'screen_name': 'Cyhra1', 'name': 'Cyhra', 'i...",3,0,257
97,1527052630169559040,1526854210033274880,2022-05-19 00:24:52,+0200,"@Totti45674455 Corona läuft nicht mehr, muss f...",de,[],1254328895932940289,ChristineFank,Fank Christine,https://twitter.com/ChristineFank/status/15270...,[],0,1,0,0,corona lang:de,"[{'screen_name': 'Totti45674455', 'name': 'Bla...",3,0,64
98,1527052605507125249,1527052605507125249,2022-05-19 00:24:47,+0200,Ich bleib corona Impfstoff frei,de,[],1499036899553464320,Claudia60140020,Jason Butterfly,https://twitter.com/Claudia60140020/status/152...,[],0,7,0,1,corona lang:de,[],3,0,31


In [21]:
df.iloc[10]["tweet"]

'@frannylicious79 Ich glaube nicht Corona ist gefährlich, sondern das Impfen! Warum gibt es soviel "Unfälle" bei den Geimpften, Geboosterten?'